In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Configuração do ambiente para utilização do Spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


# instalando a findspark
!pip install -q findspark

# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Minha Primeira Aplicação no Pyspark") \
        .getOrCreate()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Leitura dos dados

In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados/previous_application.csv')

Essa função foi criada para padronizar nomes de colunas ou textos no formato utilizado no projeto. Ela substitui todos os espaços por underscores (_) e converte as letras para maiúsculas, facilitando a leitura e mantendo consistência nos nomes usados em tabelas e arquivos.

In [ ]:
# Função para substituir espaço por underline e colocar a palavra em letras maiúsculas
import re
def tratamento_palavra(palavra):
    palavra_transformada = re.sub(r'\s', '_', palavra).upper()
    return palavra_transformada

Esse trecho de código cria uma nova lista com os valores únicos da coluna NAME_CONTRACT_STATUS do DataFrame df1, aplicando o tratamento de padronização definido anteriormente.

Primeiro, o comando df1.NAME_CONTRACT_STATUS.unique() obtém todos os status distintos dos contratos. Em seguida, o loop percorre cada valor e aplica a função tratamento_palavra(), que transforma o texto em letras maiúsculas e substitui espaços por underscores.

O resultado é uma lista (nova_lista) contendo os status de contrato padronizados — por exemplo, algo como ["ACTIVE", "CLOSED", "SOLD", "BAD_DEBT"] — pronta para ser usada em análises ou nomeação de colunas.

In [ ]:
status = df1.NAME_CONTRACT_STATUS.unique()
nova_lista = []

for palavra in status:
  nova_lista.append(tratamento_palavra(palavra))

In [ ]:
# Lê o arquivo Parquet
dados = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados/POS_CASH_balance.csv",header=True)

# Mostra os dados
dados.show()

+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|   1803195|    182943|           -31|          48.0|                 45.0|              Active|     0|         0|
|   1715348|    367990|           -33|          36.0|                 35.0|              Active|     0|         0|
|   1784872|    397406|           -32|          12.0|                  9.0|              Active|     0|         0|
|   1903291|    269225|           -35|          48.0|                 42.0|              Active|     0|         0|
|   2341044|    334279|           -35|          36.0|                 35.0|              Active|     0|         0|
|   2207092|    342166|           -32|          12.0|                 12.0|     

Esse código extrai os valores únicos da coluna NAME_CONTRACT_STATUS do DataFrame dados, padronizando-os para facilitar o uso em análises. A função tratamento_palavra() substitui espaços por underscores e converte o texto para letras maiúsculas. Em seguida, essa função é aplicada a cada valor da lista, gerando nova_lista, que contém os status dos contratos em formato limpo e padronizado, como por exemplo ["ACTIVE", "CLOSED", "BAD_DEBT"].

In [ ]:
import re

# Extrai os valores únicos diretamente para uma lista Python
status = [row["NAME_CONTRACT_STATUS"] for row in dados.select("NAME_CONTRACT_STATUS").distinct().collect()]

# Função de tratamento
def tratamento_palavra(palavra):
    return re.sub(r'\s', '_', palavra).upper()

# Aplica a transformação a cada valor único
nova_lista = [tratamento_palavra(s) for s in status]

print(nova_lista)


['DEMAND', 'APPROVED', 'COMPLETED', 'AMORTIZED_DEBT', 'RETURNED_TO_THE_STORE', 'XNA', 'ACTIVE', 'SIGNED', 'CANCELED']


## Criação de flags para nos auxiliar na visão temporal dos dados

Nesta etapa, foi habilitado o uso do SparkSQL para realizar consultas SQL diretamente sobre o DataFrame dados. O DataFrame foi registrado como uma visão temporária por meio do comando createOrReplaceTempView("dados"), permitindo tratá-lo como uma tabela SQL dentro do ambiente Spark. Em seguida, foi executada uma consulta que cria novas colunas com base no campo MONTHS_BALANCE, indicando se cada registro pertence aos últimos 3, 6, 12, 24 ou 36 meses. Essas colunas funcionam como marcadores temporais (flags) e serão úteis para análises que consideram períodos específicos, como o comportamento recente do cliente. Por fim, o resultado foi ordenado pela coluna SK_ID_PREV e salvo em uma nova visão temporária chamada df_temp_01, que servirá de base para as próximas etapas da análise.

In [ ]:
## Habilitando uso do SparkSQL
dados.createOrReplaceTempView("dados")

df_temp_01 = spark.sql("""
SELECT
    *,
      CASE
        WHEN MONTHS_BALANCE >= -3 THEN 1
        ELSE 0
    END AS ultimos_3_meses,
    CASE
        WHEN MONTHS_BALANCE >= -6 THEN 1
        ELSE 0
    END AS ultimos_6_meses,
    CASE
        WHEN MONTHS_BALANCE >= -12 THEN 1
        ELSE 0
    END AS ultimos_12_meses,
    CASE
        WHEN MONTHS_BALANCE >= -24 THEN 1
        ELSE 0
    END AS ultimos_24_meses,
    CASE
        WHEN MONTHS_BALANCE >= -36 THEN 1
        ELSE 0
    END AS ultimos_36_meses
FROM dados
ORDER BY `SK_ID_PREV`;
""")
df_temp_01.createOrReplaceTempView("df_temp_01")
df_temp_01.show()

+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+---------------+---------------+----------------+----------------+----------------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|ultimos_3_meses|ultimos_6_meses|ultimos_12_meses|ultimos_24_meses|ultimos_36_meses|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+---------------+---------------+----------------+----------------+----------------+
|   1000001|    158271|            -8|           2.0|                  0.0|           Completed|     0|         0|              0|              0|               1|               1|               1|
|   1000001|    158271|            -9|          12.0|                 11.0|              Active|     0|         0|              0|              0|               1|               1|               1|
|   100000

## Sumarizar na visão cliente (Automatizada)

O código agrupa e resume os dados por contrato (SK_ID_PREV) gerando métricas por janela temporal. Primeiro define-se a lista de colunas a agregar removendo identificadores e colunas não desejadas (SK_ID_CURR, SK_ID_PREV, MONTHS_BALANCE, NAME_CONTRACT_STATUS) e a lista de flags temporais (ultimos_3_meses, ultimos_6_meses, ultimos_12_meses, ultimos_24_meses, ultimos_36_meses). Para cada combinação de flag e coluna, são criadas expressões de agregação: para colunas que contêm DPD calcula-se apenas máximo e mínimo; para as demais calcula-se soma, média, máximo e mínimo — todas condicionais às linhas onde a flag = 1. As agregações recebem nomes padronizados com sufixo _POS_CASH e são aplicadas com .groupBy("SK_ID_PREV").agg(...), produzindo df_temp_02, uma tabela com uma linha por contrato contendo os indicadores consolidados por período, pronta para análises posteriores.

In [ ]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when

# Definir as colunas para agregação
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

colunas_flags = ['ultimos_3_meses','ultimos_6_meses','ultimos_12_meses','ultimos_24_meses','ultimos_36_meses']

expressoes_agregacao = []

for flag in colunas_flags:
  for coluna in colunas_agregacao_total:
    if 'DPD' in coluna:
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_POS_CASH"))
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_POS_CASH"))
    else:
      expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_POS_CASH"))
      expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_POS_CASH"))
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_POS_CASH"))
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_POS_CASH"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_02 = df_temp_01.groupBy("SK_ID_PREV").agg(*expressoes_agregacao).orderBy("SK_ID_PREV")

# Mostrar o DataFrame resultante
df_temp_02.show()

+----------+----------------------------------------------+----------------------------------------------+----------------------------------------------+----------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+--------------------------------------+--------------------------------------+------------------------------------------+------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+------------------------------------

O código gera agregações por contrato (SK_ID_PREV) combinando três dimensões: a categoria (valores padronizados em nova_lista, ex.: status de contrato), as flags temporais (últimos 3/6/12/24/36 meses) e as colunas de dados. Para cada combinação ele cria expressões condicionais que consideram apenas as linhas da janela (quando a flag = 1): para colunas que contêm DPD calcula máximo e mínimo; para as demais calcula soma, média, máximo e mínimo. As colunas resultantes recebem nomes padronizados com o sufixo da categoria e _POS_CASH. Por fim, todas as agregações são aplicadas com groupBy("SK_ID_PREV").agg(...), produzindo df_temp_03, uma tabela com uma linha por contrato contendo os indicadores consolidados por categoria e período. Observação: esse procedimento gera muitas colunas e pode exigir memória/tempo significativos — revise nova_lista ou as colunas agregadas se precisar reduzir o volume.

In [ ]:
# Definir as colunas para agregação
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

colunas_flags = ['ultimos_3_meses','ultimos_6_meses','ultimos_12_meses','ultimos_24_meses','ultimos_36_meses']

expressoes_agregacao = []

for categoria in nova_lista:
  for flag in colunas_flags:
    for coluna in colunas_agregacao_total:
      if 'DPD' in coluna:
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
      else:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_03 = df_temp_01.groupBy("SK_ID_PREV").agg(*expressoes_agregacao).orderBy("SK_ID_PREV")

# Mostrar o DataFrame resultante
df_temp_03.show()

+----------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+---------------------------------------------+---------------------------------------------+-------------------------------------------------+-------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+------

## Join das duas bases

Esse comando faz a junção dos dois DataFrames df_temp_02 e df_temp_03, unindo-os pela coluna comum SK_ID_PREV. Na prática, ele combina as informações consolidadas em ambos os conjuntos, reunindo em um único DataFrame (df_temp_04) todos os indicadores calculados anteriormente.

Como o join é feito pela chave de identificação do contrato, o resultado mantém uma linha por cliente, mas com todas as métricas geradas nas etapas de agregação — tanto as gerais (df_temp_02) quanto as específicas por categoria (df_temp_03). Isso permite ter uma visão completa e unificada do comportamento do cliente em diferentes contextos e períodos.

In [ ]:
df_temp_04 = df_temp_02.join(df_temp_03, "SK_ID_PREV")

## Salvar a tabela sumarizada

Nesta etapa, o DataFrame df_temp_04, que contém todas as informações consolidadas após o join, é salvo em um arquivo CSV chamado pos_cash_agg.csv. É feito processamento, exportando os dados agregados e prontos para uso em análises externas ou integração com outras etapas do projeto

In [ ]:
df_temp_04 = df_temp_02.repartition(1)
df_temp_04.write.mode("overwrite").csv("pos_cash_agg.csv",header=True)